In [40]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import pandas as pd

In [74]:
for i in range(100):

    face_path = './dataset/train/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb,1)
    
    img_show_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [75]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['img'+str(i)] = globals()['img'+str(i)][0][:17]
    globals()['img'+str(i)] = globals()['img'+str(i)].flatten()

In [76]:
train_x = np.zeros((100, 34))
for i in range(100):
    for j in range(34):
        train_x[i][j] = globals()['img'+str(i)][j]

In [77]:
for i in range(100):

    face_path = './dataset/test/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb,1)
    
    img_show_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['img_'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img_'+str(i)].append(list_points)
        globals()['img_'+str(i)] = np.array(globals()['img_'+str(i)])

In [78]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['img_'+str(i)] = globals()['img_'+str(i)][0][:17]
    globals()['img_'+str(i)] = globals()['img_'+str(i)].flatten()
    
test_x = np.zeros((100, 34))
for i in range(100):
    for j in range(34):
        test_x[i][j] = globals()['img_'+str(i)][j]

In [146]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 5:7]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 5:7]

In [84]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *

In [182]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_x.shape[1],)))
#     model.add(layers.Dropout(0.3))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(2))
    
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    return model

model = build_model()
model.fit(train_x, train_y, epochs=100, verbose=0)

ppredict = model.predict(test_x)

ppredict_df = pd.DataFrame(ppredict)

ppredict_df

4/4 [==============================] - 0s 1ms/step


,0,1
0,0.867459,-4.851773
1,0.201779,-2.330559
2,0.282208,-2.707444
3,0.959553,1.289935
4,0.363537,-1.632186
...,...,...
95,0.431721,-3.348240
96,0.564068,-2.511572
97,0.760502,2.796937
98,0.260426,-3.235741


In [170]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=150, random_state=42)
model.fit(train_x, train_y)

pred = model.predict(test_x)
pred_df = pd.DataFrame(pred)
pred_df.columns = ['jawsPosition','jawsSize']
pred_df

,jawsPosition,jawsSize
0,0.499486,0.503956
1,0.534185,0.665173
2,0.531494,0.484562
3,0.449787,0.510031
4,0.587859,0.670883
...,...,...
95,0.466608,0.637656
96,0.534939,0.459401
97,0.484920,0.460017
98,0.412298,0.525094


In [165]:
test_y

,jawsPosition,jawsSize
0,0.323832,0.595424
1,0.252126,0.766253
2,0.466237,0.531876
3,0.289253,0.519737
4,0.225629,0.502349
...,...,...
95,0.320538,0.766127
96,0.530781,0.438227
97,0.565084,0.416299
98,0.569411,0.301160


In [166]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(test_y, pred)**0.5
print(rmse)

0.16866991583803492
